# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (sparse implementation)

In [ ]:
import logging
import numpy as np
import h5py
import jax
from fastdla.sparse_pauli_sum import SparsePauliSumArray, SparsePauliSum
from fastdla.lie_closure import lie_closure, orthogonalize
from fastdla.generators.z2lgt_hva import (z2lgt_hva_generators, z2lgt_gauss_projector,
                                          z2lgt_u1_projector, z2lgt_translation_projector,
                                          z2lgt_symmetry_eigenspace)

logging.basicConfig(level=logging.WARNING)
logging.getLogger('fastdla').setLevel(logging.INFO)
jax.config.update('jax_enable_x64', True)
SparsePauliSum.switch_impl('fast')

In [ ]:
data_file = ''
max_workers = 48

### Choice of symmetry sector

In [ ]:
num_fermions = 4
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1, 1, -1]
u1_total_charge = 0
t_jphase = 0

### Full list of HVA generators

In [ ]:
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

### Using SparsePauliSums

In [ ]:
# Compute the DLA of the full space
dla_full = lie_closure(generators_full, min_tasks=200, max_workers=max_workers)
print(f'DLA dimension is {len(dla_full)}')

In [ ]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, u1_total_charge)
t_projector = z2lgt_translation_projector(num_fermions, t_jphase)
symm_projector = gauss_projector @ u1_projector @ t_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators_symm = SparsePauliSumArray([(op @ symm_projector).normalize() for op in generators_full])

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
basis = SparsePauliSumArray([generators_symm[0]])
for gen in generators_symm[1:]:
    orth = orthogonalize(gen, basis)
    if orth.num_terms != 0:
        basis.append(orth.normalize())
print(f'{len(basis)} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla = lie_closure(basis, min_tasks=200, max_workers=max_workers)
print(f'Subspace DLA dimension is {len(dla)}')

### Using matrices

In [ ]:
generators_csr = [gen.to_matrix(sparse=True) for gen in generators_full]
symm_eigenspace = z2lgt_symmetry_eigenspace(gauss_eigvals, u1_total_charge, t_jphase)
generators_reduced = np.array([symm_eigenspace.conjugate().T @ gen.dot(symm_eigenspace)
                               for gen in generators_csr])
normsq = np.einsum('gij,gji->g', generators_reduced.conjugate(), generators_reduced)
normsq /= generators_reduced.shape[-1]
norm = np.sqrt(normsq)
generators_reduced /= norm[:, None, None]

In [ ]:
# Refine the generators list in case some are linearly dependent in the subspace
basis = np.array([generators_reduced[0]])
for igen, gen in enumerate(generators_reduced[1:]):
    orth = orthogonalize(gen, basis)
    norm = np.sqrt(np.trace(orth.conjugate().T @ orth) / orth.shape[0])
    if not np.isclose(norm, 0.):
        basis = np.concatenate([basis, orth[None, ...] / norm], axis=0)
print(f'{basis.shape[0]} generators are independent')

In [ ]:
# Compute the DLA of the subspace
dla_arr = lie_closure(basis)
print(f'Subspace DLA dimension is {len(dla_arr)}')

### Save the calculation result

In [ ]:
if data_file:
    with h5py.File(data_file, 'a') as out:
        if f'nf={num_fermions}' not in out:
            nf_group = out.create_group(f'nf={num_fermions}')
            for gname, oplist in [
                ('generators_full', generators_full),
                ('dla_full', dla_full),
                ('generators_symm', generators_symm),
                ('dla_symm', dla)
            ]:
                group = nf_group.create_group(gname)
                group.create_dataset('indices', data=oplist.indices)
                group.create_dataset('coeffs', data=oplist.coeffs)
                group.create_dataset('indptr', data=oplist.ptrs)

            nf_group.create_dataset('dla_symm/gauss_eigvals', data=gauss_eigvals)
            nf_group.create_dataset('dla_symm/u1_total_charge', data=u1_total_charge)
            nf_group.create_dataset('dla_symm/t_jphase', data=t_jphase)